In [1]:
import pandas as pd
import sqlite3

## Instancia de BBDD

In [2]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/Bar_TB.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

### Funcion para interactuar con la base de datos

In [3]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

## Crear las tablas

In [4]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

### Tabla Cerveza

In [5]:
query_cerveza = """
CREATE TABLE cerveza (
    CodC VARCHAR(3),
    Envase VARCHAR(50),
    Capacidad FLOAT,
    Stock INT
) 
"""

crsr.execute(query_cerveza)


### Tabla Bares

In [6]:
query_bares = """
CREATE TABLE bares (
    CodB VARCHAR(4),
    Nombre VARCHAR(50),
    Cif VARCHAR(10),
    Localidad VARCHAR(50)
)    
"""

crsr.execute(query_bares)

### Tabla Empleados

In [7]:
query_empleados = """
CREATE TABLE empleados (
    CodE VARCHAR(2),
    Nombre VARCHAR(50),
    Sueldo INT
)    
"""

crsr.execute(query_empleados)

### Tabla Reparto

In [8]:
query_reparto = """
CREATE TABLE reparto (
    CodE VARCHAR(2),
    CodB VARCHAR(4),
    CodC VARCHAR(3),
    Fecha DATE,
    Cantidad INT
)    
"""

crsr.execute(query_reparto)

## Insertar Datos

In [9]:
# Insertamos datos en la tabla cerveza
query_insert = """
INSERT INTO cerveza VALUES
('01', 'Botella', 0.2, 3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04', 'Botella', 1, 288),
('05', 'Barril', 60, 30)
"""
crsr.execute(query_insert)

In [10]:
# Insertamos datos en la tabla reparto
query_insert2 = """
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) VALUES
('1','001','01','2005-10-21',240),
('1','001','02','2005-10-21',48),
('1','002','03','2005-10-22',60),
('1','004','05','2005-10-22',4),
('2','002','03','2005-10-22',48),
('2','002','05','2005-10-23',2),
('2','004','00','2005-10-23',480),
('2','004','02','2005-10-24',72),
('3','003','03','2005-10-24',48),
('3','003','04','2005-10-25',20)
"""

crsr.execute(query_insert2)

In [11]:
# Insertamos datos en la tabla Bares

bares=pd.DataFrame({
                    "CodB": ["001","002","003","004"],
                    "Nombre": ["Stop","Las Vegas","Club Social","Otra Ronda"],
                    "Cif": ["11111111X","22222222Y",None,"33333333Z"],
                    "Localidad":["Villa Botijo","Villa Botijo","Las Ranas","La Esponja"]
                    })
bares

bares.to_sql(name='bares',con=connection,if_exists="append",index=False)

4

In [12]:
# Insertamos datos en la tabla Bares

empleados=pd.DataFrame({
                    "CodE": ["1","2","3"],
                    "Nombre": ["Prudencio Caminero","Vicente Merario","Valentin Siempre"],
                    "Sueldo": [120000,110000,100000],
                    })
empleados

empleados.to_sql(name='empleados',con=connection,if_exists="append",index=False)

3

In [13]:
connection.commit()

In [14]:
connection.close()

## Respuestas

In [15]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/Bar_TB.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

In [16]:
#1 Obtener  el  nombre    de  los  empleados   que  hayan  repartido  al  bar  Stop  durante la semana del 17 al 23 de octubre de 2005
query1 = """
SELECT DISTINCT
e.Nombre
FROM empleados e
INNER JOIN reparto r
ON e.CodE = r.CodE
INNER JOIN bares b
ON b.CodB = r.CodB
WHERE b.Nombre = 'Stop' AND r.Fecha BETWEEN '2005-10-17' AND '2005-10-23'
"""
sql_query(query1)

,Nombre
0,Prudencio Caminero
